In [1]:
# Existing category mapping
category_mapping = {
    "Wordpress": "Wordpress Development",
    "Finance": "Finance",
    "BBA": "Business Administration",
    "HR": "Human Resources",
    "Accounting": "Accounting",
    "Marketing": "Marketing",
    "Sales": "Sales",
    "Graphic Designer": "Graphic Designer",
    "Customer Service": "Customer Service",
    "Project Management": "Project Management",
    "Operations": "Operations",
    "Video Editing": "Graphic Designer",
    "Legal": "Legal",
    "Healthcare": "Healthcare",
    "Python": "Data Science - Python",
    "Machine Learning": "Data Science - Machine Learning",
    "Data Visualization": "Data Science - Data Visualization",
    "Data Engineering": "Data Science - Data Engineering",
    "Big Data Analytics": "Data Science - Big Data Analytics",
    "Data Mining": "Data Science - Data Mining",
    "Predictive Analytics": "Data Science - Predictive Analytics",
    "Statistical Analysis": "Data Science - Statistical Analysis",
    "Java": "Software Development - Java",
    "C++": "Software Development - C++",
    "Frontend Development": "Frontend Development",
    "Backend Development": "Backend Development",
    "Mobile App Development": "Mobile App Development",
    "Game Development": "Software Development - Game Development",
    "DevOps": "Software Development - DevOps",
    "Embedded Systems": "Software Development - Embedded Systems",
    "Cloud Computing": "Software Development - Cloud Computing",
    "Deep Learning": "Machine Learning Engineer - Deep Learning",
    "Html": "Web Development - HTML",
    "php": "Web Development - PHP",
    "Natural Language Processing (NLP)": "NLP Engineer",
    "Computer Vision": "Computer Vision Engineer",
    "Reinforcement Learning": "Machine Learning Engineer",
    "Time Series Analysis": "Machine Learning Engineer",
    "Neural Networks": "Machine Learning Engineer",
    "Finance": "Finance - General",
    "BBA": "Finance - BBA",
    "HR": "Human Resources",
    "Accounting": "Accounting",
    "Marketing": "Marketing - General",
    "Sales": "Sales - General",
    "Customer Service": "Customer Service - General",
    "Project Management": "Project Management - General",
    "Operations": "Operations - General",
    "Legal": "Legal - General",
    "Healthcare": "Healthcare - General",
    "Social Media Management": "Social Media - Management",
    "Social Media Marketing": "Social Media - Marketing",
    "Social Media Content Creation": "Social Media - Content Creation",
    "Social Media Community Engagement": "Social Media - Community Engagement",
    "Social Media Analytics": "Social Media - Analytics",
    "Financial Analysis": "Finance - Financial Analysis",
    "Investment Banking": "Finance - Investment Banking",
    "Corporate Finance": "Finance - Corporate Finance",
    "Financial Risk Management": "Finance - Financial Risk Management",
    "Financial Modeling": "Finance - Financial Modeling",
    "Portfolio Management": "Finance - Portfolio Management",
    "Medical Research": "Healthcare - Medical Research",
    "Healthcare Administration": "Healthcare - Administration",
    "Clinical Research": "Healthcare - Clinical Research",
    "Medical Coding": "Healthcare - Medical Coding",
    "Medical Billing": "Healthcare - Medical Billing",
    "Health Informatics": "Healthcare - Health Informatics",
    "Digital Marketing": "Marketing - Digital Marketing",
    "Email Marketing": "Marketing - Email Marketing",
    "Search Engine Optimization (SEO)": "Marketing - SEO",
    "Content Management": "Marketing - Content Management",
    "Business Analysis": "Business Administration - Business Analysis",
    "Entrepreneurship": "Business Administration - Entrepreneurship",
    "Supply Chain Management": "Operations - Supply Chain Management",
    "Logistics": "Operations - Logistics",
    "Quality Management": "Operations - Quality Management",
    "Production Planning": "Operations - Production Planning",
    "Contract Law": "Legal - Contract Law",
    "Corporate Law": "Legal - Corporate Law",
    "Employment Law": "Legal - Employment Law",
}


In [ ]:
import imaplib
import email
import os
import re
import warnings
import joblib
import spacy
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from PyPDF2 import PdfReader
import docx
from nltk.corpus import stopwords
import openai
import json



# Email credentials
EMAIL = 'maimoonakhan673763@outlook.com'
PASSWORD = 'USE YOUR PASSWORD HERE'

# Define the IMAP server details
IMAP_SERVER = 'imap-mail.outlook.com'
IMAP_PORT = 993

# Define the keywords to search for in the subject line
SEARCH_KEYWORDS = [
    'Application', 'Job', 'Internship', 'Seeking Opportunity', 'Resume', 'Candidate',
    'Applying for', 'Opportunity', 'Seeking', 'Seeking Internship', 'Seeking Internship Opportunity','For job','For'
]

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

# Load the trained model and vectorizer for category prediction
model = joblib.load('trained_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')  # Load the fitted vectorizer

# Load Spacy model for NER
nlp = spacy.load("en_core_web_sm")

# Skills list
SKILLS = [
    "Python", "Java", "C++", "JavaScript", "HTML", "CSS", "SQL", "Machine Learning", "Data Analysis",
    "Data Visualization", "Statistics", "Project Management", "Team Leadership", "Agile Methodologies",
    "Scrum", "Business Analysis", "Market Research", "Financial Analysis", "Digital Marketing",
    "Content Marketing", "Social Media Marketing", "Search Engine Optimization (SEO)", "User Experience (UX) Design",
    "User Interface (UI) Design", "Graphic Design", "Web Development", "Mobile App Development",
    "Cloud Computing", "Network Security", "System Administration", "DevOps", "Technical Writing",
    "Content Writing", "Copywriting", "Editing", "Proofreading", "Translation", "Customer Service", "Sales",
    "Account Management", "Public Speaking", "Negotiation", "Time Management", "Problem Solving",
    "Critical Thinking", "Creativity"
]

# Lists for certificates, projects, and experience keywords
CERTIFICATES = ["AWS Certified", "Google Cloud Certified", "PMP", "Six Sigma", "Certified ScrumMaster", "CCNA", "Microsoft Certified"]
PROJECTS_KEYWORDS = ["Machine Learning", "NLP", "Python", "Java", "C++", "Data Science"]
EXPERIENCE_KEYWORDS = ["year", "worked", "years", "months", "lead", "freelance"]

# GPT-3 API key
openai.api_key = 'USE YOUR API KEY HERE'

# Function to rank resumes using GPT-3
def rank_resumes_with_gpt3(resumes):
    rankings = []
    for resume in resumes:
        # Define the prompt for GPT-3
        prompt = f"Rank the following resume:\n{resume}\n\nBased on the provided resume, please provide a ranking on a scale of 1 to 10, with 10 being the highest."

        # Request ranking from GPT-3
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            temperature=0.7
        )

        # Extract ranking from GPT-3 response
        ranking = response.choices[0].message['content'].strip()
        rankings.append((resume, ranking))
    return rankings

# Function to send email
def send_email(recipient_email, subject, message):
    try:
        # Set up SMTP server
        server = smtplib.SMTP('smtp-mail.outlook.com', 587)
        server.starttls()
        
        # Login to your email account
        server.login(EMAIL, PASSWORD)

        # Create message container
        msg = MIMEMultipart()
        msg['From'] = EMAIL
        msg['To'] = recipient_email
        msg['Subject'] = subject

        # Add message body
        msg.attach(MIMEText(message, 'plain'))

        # Send the message via the SMTP server
        server.send_message(msg)
        
        # Clean up the SMTP server
        server.quit()
        
        print(f"Email sent successfully to {recipient_email}")
        
    except smtplib.SMTPException as e:
        print("SMTP Error:", e)
    except Exception as e:
        print("Error sending email:", e)

# Connect to the IMAP server
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
try:
    mail.login(EMAIL, PASSWORD)
except imaplib.IMAP4_SSL.error as e:
    print("Error logging in to the IMAP server:", e)
    exit()

try:
    mail.select('inbox')
    for keyword in SEARCH_KEYWORDS:
        result, data = mail.search(None, f'(SUBJECT "{keyword}")')
        if result == 'OK':
            break
    else:
        raise Exception("No relevant emails found")

except Exception as e:
    print("Error:", e)
    mail.logout()
    exit()

# Function to extract text from Word documents
def extract_text_from_docx(file_path):
    try:
        doc = docx.Document(file_path)
        full_text = []
        for paragraph in doc.paragraphs:
            full_text.append(paragraph.text)
        return '\n'.join(full_text)
    except Exception as e:
        print(f"Error extracting text from Word document {file_path}: {e}")
        return None

def extract_ranking(response_text):
    # Try to extract ranking as an integer
    try:
        ranking = int(response_text)
        return ranking
    except ValueError:
        pass

    # Extract numerical ranking using regular expression
    match = re.search(r'\b(\d+)\b', response_text)
    if match:
        return int(match.group(0))

    # Try to extract ranking from textual representation
    ranking_mapping = {
        "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
        "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10
    }
    for word in response_text.lower().split():
        if word in ranking_mapping:
            return ranking_mapping[word]
    
    # Try to extract overall ranking from the response
    overall_match = re.search(r'Overall Ranking:\s*(\d+)', response_text)
    if overall_match:
        return int(overall_match.group(1))
    else:
        # If extraction fails, return None
        return None

# Function to extract candidate information from text
def extract_candidate_info(text):
    candidate_info = {}
    phone_found = False 

    lines = text.split('\n')

    for line in lines:
        email_check = line.replace(" ", "")
        match_email = re.search(r'[\w\.-]+@[\w\.-]+com', line)
        space_match_email = re.search(r'[\w\.-]+@[\w\.-]+com', email_check)

        if space_match_email and match_email:
            space_match_email = space_match_email.group()
            match_email = match_email.group()
            if len(space_match_email) < len(match_email):
                candidate_info['Email'] = space_match_email
            else:
                candidate_info['Email'] = match_email
        elif(space_match_email):
            candidate_info['Email'] = space_match_email.group()
        elif(match_email):
            candidate_info['Email'] = match_email.group()

        space_match_phone = line.replace("(", "").replace(")", "").replace(" ", "")
        match_phone = re.search(r'(\+?\d{1,3}[-\.\s]??\d{3,4}[-\.\s]??\d{4}|\(\+\d{1,3}\)\s*\d{3,4}[-\.\s]??\d{4}|\+\d{1,3}[-\.\s]??\d{3,4})', space_match_phone)
        if match_phone:
            # Check if the matched string is likely a phone number
            if len(match_phone.group().replace('-', '')) >= 10: 
                candidate_info['Phone'] = match_phone.group()
                phone_found = True 
        # Extract candidate name (assuming it's the first line)
        if 'NAME' in line:
            candidate_info['Name'] = line.split('NAME')[1].strip()

    if not phone_found:
        candidate_info['Phone'] = 'Null'
    return candidate_info

# Initialize candidate counter
candidate_count = 1
total_resumes = 0
progressed_resumes = 0
high_rank_resumes = 0
low_rank_resumes = 0

# Process each email
if data:
    email_nums = data[0].split()
    for num in email_nums:
        try:
            result, email_data = mail.fetch(num, '(RFC822)')
            if result != 'OK':
                raise Exception("Failed to fetch email data")

            raw_email = email_data[0][1]
            msg = email.message_from_bytes(raw_email)

            if msg.get_content_maintype() == 'multipart':
                for part in msg.walk():
                    if part.get_content_maintype() == 'multipart':
                        continue
                    if part.get('Content-Disposition') is None:
                        continue

                    filename = part.get_filename()
                    if filename:
                        with open(filename, 'wb') as f:
                            f.write(part.get_payload(decode=True))

                        try:
                            if filename.endswith('.pdf'):
                                with open(filename, 'rb') as file:
                                    pdf_reader = PdfReader(file)
                                    resume_text = ''
                                    for page_num in range(len(pdf_reader.pages)):
                                        page = pdf_reader.pages[page_num]
                                        resume_text += page.extract_text()
                            elif filename.endswith('.docx'):
                                resume_text = extract_text_from_docx(filename)
                            else:
                                continue

                            if resume_text:
                                # Increment candidate counter
                                candidate_name = f"Candidate {candidate_count}"
                                candidate_count += 1

                                # Extract candidate information
                                candidate_info = extract_candidate_info(resume_text)

                                # Rank the resume using GPT-3
                                rankings = rank_resumes_with_gpt3([resume_text])

                                # Extract skills from the resume
                                skills = [skill for skill in SKILLS if skill in resume_text]

                                # Determine category based on skills
                                resume_category = "Unknown"
                                for skill in skills:
                                    if skill in category_mapping:
                                        resume_category = category_mapping[skill]
                                        break

                                # Predict category using the trained model
                                cleaned_resume_text = " ".join([word for word in resume_text.split() if word.lower() not in stopwords.words('english')])
                                resume_vector = vectorizer.transform([cleaned_resume_text])
                                predicted_category = model.predict(resume_vector)[0]

                                # Print resume rankings and send emails
                                for resume_text, ranking_text in rankings:
                                    # Extract candidate's information from the email
                                    print("-" * 50)  # Add a separator line
                                    candidate_email = candidate_info.get('Email', 'Unknown')
                                    print(candidate_name)
                                    print("Email:", candidate_email)
                                    print("Resume Category (Predicted Skills):", resume_category)
                                    print("Ranking Text:", ranking_text)
                                    # Extract numerical ranking
                                    ranking = extract_ranking(ranking_text)
                                    if ranking is not None:
                                        print("Ranking:", ranking)
                                        if ranking >= 7:
                                            subject = "Congratulations on Your Selection"
                                            message = f"""\
Dear Candidate,

Congratulations! We are pleased to inform you that your resume for the '{predicted_category}' position has achieved a high ranking. You have been selected to proceed further in our selection process. We will contact you shortly with more details.

Thank you for your application and interest in our company.

Best regards,
HMA TECH SOLUTIONS
"""
                                        else:
                                            subject = "Unfortunately for this position"
                                            message = f"""\
Dear Candidate,

We regret to inform you that your resume for the '{predicted_category}' position did not achieve the expected ranking. We appreciate your interest in our company and encourage you to continue pursuing opportunities.

Thank you for your application.

Best regards,
HMA TECH SOLUTIONS
"""

                                        # Update resume statistics
                                        total_resumes += 1
                                        if ranking >= 7:
                                            progressed_resumes += 1
                                            high_rank_resumes += 1
                                        else:
                                            progressed_resumes += 1
                                            low_rank_resumes += 1

                                        # Send email if the recipient email is valid
                                        if candidate_email != 'Unknown':
                                            send_email(candidate_email, subject, message)
                                    else:
                                        print("Unable to extract numerical ranking from the response.")
                        except Exception as e:
                            print(f"Error processing document {filename}: {e}")
                            continue
        except Exception as e:
            print("Error processing email:", e)
            continue
else:
    print("No emails found in the inbox.")

# Print summary of processed resumes
print("Total resumes processed:", total_resumes)
print("Resumes progressed:", progressed_resumes)
print("High-ranked resumes:", high_rank_resumes)
print("Low-ranked resumes:", low_rank_resumes)

# Save summary data to resume_data.json
resume_data = {
    "Total resumes processed": total_resumes,
    "Resumes progressed": progressed_resumes,
    "High-ranked resumes": high_rank_resumes,
    "Low-ranked resumes": low_rank_resumes
}

with open('resume_data.json', 'w') as json_file:
    json.dump(resume_data, json_file)

print("Summary data saved to resume_data.json")

--------------------------------------------------
Candidate 1
Email: maimoonakhan673763@gmail.com
Resume Category (Predicted Skills): Data Science - Python
Ranking Text: I would rank this resume as a 7 out of 10. 

Strengths:
1. Clearly defined objectives and skills in front-end development, logo design, machine learning, and data science.
2. Demonstrated experience in building a health predicting
Ranking: 7
Email sent successfully to maimoonakhan673763@gmail.com
--------------------------------------------------
Candidate 2
Email: hello@reallygreatsite.com
Resume Category (Predicted Skills): Project Management - General
Ranking Text: I would rate this resume a 7. 

The resume includes relevant sections such as education, work experience, skills, and contact information. The candidate's education is clearly outlined, showcasing their academic achievements and language proficiency. The work experience section highlights the
Ranking: 7
Email sent successfully to hello@reallygreatsite.co